A simple example of text being 

In [ ]:
import cadquery as cq
import openmc
import paramak
import paramak_neutronics

text = cq.Workplane().text( 
             txt="The Paramak can easily convert CAD\nto neutronics models for simulations.",
             fontsize=0.5,
             distance=-1,
             cut=True,
             halign="left",
             valign="bottom",
             font="Sans"
)

# this creates an empty Shape and populates it with the CQ object
my_shape = paramak.Shape()
my_shape.solid = text
my_shape.method = 'pymoab'
my_shape.stl_filename = 'text.stl'
my_shape.material_tag = 'my_material'
my_shape.export_stp('text_cad.stp')

coords = openmc.stats.Point((4, 1, 0.5))
energy = openmc.stats.Discrete([1.41E7], [1.0])
source = openmc.Source(space=coords, energy=energy)

my_model = paramak_neutronics.NeutronicsModel(
    h5m_filename=my_shape.export_h5m(),
    source=source,
    materials={'my_material': 'Li4SiO4'},
    mesh_tally_3d=['heating', '(n,Xt)'],
    simulation_batches=2,  # to get a good mesh tally result with will need increasing
    mesh_3d_resolution=(850, 350, 10),
    mesh_3d_corners=[(0, -1, -1), (8.5, 2.5, 0.5)],
)

my_model.simulate()
